In [1]:
from pathlib import Path
import random

### files number check (audios vs. sections)

In [47]:
input_path = Path("/home/hbli/songformer/repo/SongFormer/runs/hx_train_hooktheory_v2/results/hooktheory_all.scp")
train_path = Path("/home/hbli/songformer/repo/SongFormer/runs/hx_train_hooktheory_v2/results/hooktheory_train.scp")
test_path  = Path("/home/hbli/songformer/repo/SongFormer/runs/hx_train_hooktheory_v2/results/hooktheory_test.scp")

n_test = 1000
seed = 42

In [48]:
lines = input_path.read_text(encoding="utf-8").splitlines()
lines = [ln for ln in lines if ln.strip() != ""]

total = len(lines)

# sampling
rng = random.Random(seed) if seed is not None else random.Random()
test_indices = set(rng.sample(range(total), n_test))

test_lines = [lines[i] for i in range(total) if i in test_indices]
train_lines  = [lines[i] for i in range(total) if i not in test_indices]

In [49]:
len(train_lines), len(test_lines), total

(12888, 1000, 13888)

In [40]:
# 确认1000个样本是否cover全部label类型
def extract_label_from_line(line):
    line = line.strip()
    if not line:
        return None

    parts = line.split()
    if len(parts) < 3:
        return None
    if len(parts) > 3:
        return 'several'

    label = parts[2]
    return label if label else None

test_lines_sec = []
for p in test_lines:
    pp = Path(p)
    new_dir = str(pp.parent).replace("打包结果_audio", "打包结果_section")
    new_name = pp.stem + "_measure.sec"
    test_lines_sec.append(str(Path(new_dir) / new_name))

unique_labels = set()
for fp in test_lines_sec:
    fp = Path(fp)
    with fp.open("r", encoding = "utf-8", errors = "ignore") as f:
        for line in f:
            label = extract_label_from_line(line)
            unique_labels.add(label)

In [43]:
len(unique_labels)

9

### 输出train/test scp

In [46]:
train_path.write_text("\n".join(train_lines) + "\n", encoding="utf-8")
test_path.write_text("\n".join(test_lines) + "\n", encoding="utf-8")

67611